In [ ]:
import calendar
from dotenv import load_dotenv
import os

import requests
import pandas as pd

from src.paths import PARENT_DIR, RAW_DATA_DIR

load_dotenv(PARENT_DIR / ".env")
EIA_API_KEY = os.environ["EIA_API_KEY"]


def download_raw_data(year: int, month: int):
    file_path = RAW_DATA_DIR / f"demand_{year}_{month}.csv"
    if file_path.exists():
        print(f"File demand_{year}_{month}.csv exists locally already, try next URL")
    else:
        # Need the number of days in the current (year, month)
        _, num_days = calendar.monthrange(year, month)    

        URL = (
            "https://api.eia.gov/v2/electricity/rto/daily-region-data/data/"
            "?frequency=daily"
            "&data[0]=value"
            "&facets[timezone][]=Eastern"
            "&facets[type][]=D"
            f"&start={year}-{month:02d}-01"
            f"&end={year}-{month:02d}-{num_days}"
            "&sort[0][column]=period"
            "&sort[0][direction]=desc"
            "&offset=0"
            "&length=5000"
            f"&api_key={EIA_API_KEY}"
        )

        response = requests.get(url=URL).json()["response"]["data"]
        data = pd.DataFrame(response)

        # Tidies dataframe and saves to csv
        data = data[["period", "respondent", "value"]].copy()
        data.rename(
            columns={
                "period": "datetime",
                "value": "demand",
                "respondent": "ba_code",
            },
            inplace=True,
        )

        data.to_csv(file_path, index=False)
        print(f"Data for {year}_{month} successfully downloaded to demand_{year}_{month}.csv")



In [3]:
# Download raw data from 01 October 2022 to 31 October 2024, about two years worth
year = 2022
for month in range(10, 13):
    download_raw_data(year, month)

print(f"\nData for {year} finished downloading\n")

year = 2023
for month in range(1, 13):
    download_raw_data(year, month)

print(f"\nData for {year} finished downloading\n")

year = 2024
for month in range(1, 11):
    download_raw_data(year, month)

print(f"\nData for {year} finished downloading\n")

Data for 2022_10 successfully downloaded to demand_2022_10.csv
Data for 2022_11 successfully downloaded to demand_2022_11.csv
Data for 2022_12 successfully downloaded to demand_2022_12.csv

Data for 2022 finished downloading

File demand_2023_1.csv exists locally already, try next URL
File demand_2023_2.csv exists locally already, try next URL
File demand_2023_3.csv exists locally already, try next URL
File demand_2023_4.csv exists locally already, try next URL
File demand_2023_5.csv exists locally already, try next URL
File demand_2023_6.csv exists locally already, try next URL
File demand_2023_7.csv exists locally already, try next URL
File demand_2023_8.csv exists locally already, try next URL
File demand_2023_9.csv exists locally already, try next URL
File demand_2023_10.csv exists locally already, try next URL
File demand_2023_11.csv exists locally already, try next URL
File demand_2023_12.csv exists locally already, try next URL

Data for 2023 finished downloading

File demand_20